# Training ANN model on data

    Find best split number

In [9]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

In [ ]:
split_range = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


for i in split_range:
    # import data
    data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

    # 2018 data
    data = data.loc[data.index > 2018090000, :]

    # reset index
    data.reset_index(inplace = True)
    data.drop('index', axis = 1, inplace = True)

    # Divide features and labels
    X = data.iloc[:, 0:15]
    y = data.loc[:, 'Offers']
    
    X.fillna(method = 'ffill', inplace = True)
    y.fillna(method = 'ffill', inplace = True)

    X = X.round(20)
    
    # divide data into train and test with 10% test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, shuffle=False)

    # feature scaling
    sc_X = MinMaxScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
    
    def regressor_tunning(n_hidden = 5, 
                          n_neurons = 40, 
                          kernel_initializer = "he_normal",
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(Dense(units = n_neurons, input_dim = 15))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
        for layer in range(n_hidden):
            model.add(Dense(n_neurons))
            model.add(keras.layers.LeakyReLU(alpha = 0.2))
            model.add(Dropout(rate = 0.1))
        model.add(Dense(units = 1, activation = 'linear'))
        optimizer = optimizers.Adamax(lr = 0.001)
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
        return model

    tscv = TimeSeriesSplit(n_splits = i)
    hist_list = pd.DataFrame()
    count = 1
    
    regressor = regressor_tunning()
    
    for train_index, test_index in tscv.split(X_train):
          X_train_split, X_test_split = X_train[train_index], X_train[test_index]
          y_train_split, y_test_split = y_train[train_index], y_train[test_index]
          hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 100)
          hist_list = hist_list.append(hist.history, ignore_index = True)
          print(count)
          count = count + 1

    a = []
    b = []
    
    for i in range(len(hist_list.mse)):
        a.append(np.mean(hist_list.mse[i]))
        b.append(np.mean(hist_list.mae[i]))
      
    mse_cv.append(np.mean(a))
    mae_cv.append(np.mean(b))
    
    # predict for X_test  
    y_pred = regressor.predict(X_test)

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525
    
    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
    
    # smal adjustment
    y_test.replace(0, 0.0001,inplace = True)

    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)


C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/100
67/67 [==============================] - 0s 2ms/step - loss: 16617.1152 - mse: 16617.1152 - mae: 114.7932
Epoch 2/100
67/67 [==============================] - 0s 1ms/step - loss: 6698.7241 - mse: 6698.7241 - mae: 55.3658
Epoch 3/100
67/67 [==============================] - 0s 1ms/step - loss: 3948.5432 - mse: 3948.5432 - mae: 33.5642
Epoch 4/100
67/67 [==============================] - 0s 1ms/step - loss: 4061.4592 - mse: 4061.4592 - mae: 32.3877
Epoch 5/100
67/67 [==============================] - 0s 1ms/step - loss: 3809.1660 - mse: 3809.1660 - mae: 32.2976
Epoch 6/100
67/67 [==============================] - 0s 1ms/step - loss: 3785.7117 - mse: 3785.7117 - mae: 31.8002
Epoch 7/100
67/67 [==============================] - 0s 1ms/step - loss: 3948.5559 - mse: 3948.5554 - mae: 32.2700
Epoch 8/100
67/67 [==============================] - 0s 2ms/step - loss: 3723.4800 - mse: 3723.4800 - mae: 31.7067
Epoch 9/100
67/67 [==============================] - 0s 2ms/step - loss: 4000

67/67 [==============================] - 0s 1ms/step - loss: 3611.7148 - mse: 3611.7148 - mae: 30.0130
Epoch 72/100
67/67 [==============================] - 0s 1ms/step - loss: 3862.2678 - mse: 3862.2683 - mae: 30.3137
Epoch 73/100
67/67 [==============================] - 0s 1ms/step - loss: 3654.0181 - mse: 3654.0181 - mae: 30.0587
Epoch 74/100
67/67 [==============================] - 0s 1ms/step - loss: 3679.3652 - mse: 3679.3652 - mae: 29.1129
Epoch 75/100
67/67 [==============================] - 0s 2ms/step - loss: 3604.7681 - mse: 3604.7681 - mae: 29.9215
Epoch 76/100
67/67 [==============================] - 0s 1ms/step - loss: 3768.0078 - mse: 3768.0078 - mae: 30.3154
Epoch 77/100
67/67 [==============================] - 0s 1ms/step - loss: 3697.1021 - mse: 3697.1021 - mae: 30.0527
Epoch 78/100
67/67 [==============================] - 0s 1ms/step - loss: 3599.8445 - mse: 3599.8445 - mae: 30.1445
Epoch 79/100
67/67 [==============================] - 0s 1ms/step - loss: 3615.1873 -

133/133 [==============================] - 0s 2ms/step - loss: 2606.8186 - mse: 2606.8186 - mae: 28.2244
Epoch 42/100
133/133 [==============================] - 0s 1ms/step - loss: 2587.7219 - mse: 2587.7219 - mae: 28.3470
Epoch 43/100
133/133 [==============================] - 0s 1ms/step - loss: 2570.6882 - mse: 2570.6882 - mae: 28.0277
Epoch 44/100
133/133 [==============================] - 0s 2ms/step - loss: 2616.9399 - mse: 2616.9399 - mae: 28.6844
Epoch 45/100
133/133 [==============================] - 0s 2ms/step - loss: 2598.8572 - mse: 2598.8572 - mae: 28.5663
Epoch 46/100
133/133 [==============================] - 0s 2ms/step - loss: 2601.1860 - mse: 2601.1860 - mae: 28.4874
Epoch 47/100
133/133 [==============================] - 0s 1ms/step - loss: 2558.8403 - mse: 2558.8403 - mae: 28.3785
Epoch 48/100
133/133 [==============================] - 0s 1ms/step - loss: 2630.0830 - mse: 2630.0830 - mae: 28.8386
Epoch 49/100
133/133 [==============================] - 0s 2ms/step -

200/200 [==============================] - 0s 2ms/step - loss: 2132.3032 - mse: 2132.3032 - mae: 27.4829
Epoch 11/100
200/200 [==============================] - 0s 2ms/step - loss: 2207.2676 - mse: 2207.2676 - mae: 27.9893
Epoch 12/100
200/200 [==============================] - 0s 2ms/step - loss: 2173.9634 - mse: 2173.9634 - mae: 27.6862
Epoch 13/100
200/200 [==============================] - 0s 2ms/step - loss: 2169.4895 - mse: 2169.4895 - mae: 27.5448
Epoch 14/100
200/200 [==============================] - 0s 2ms/step - loss: 2171.3345 - mse: 2171.3345 - mae: 27.4506
Epoch 15/100
200/200 [==============================] - 0s 2ms/step - loss: 2183.0400 - mse: 2183.0400 - mae: 27.5770
Epoch 16/100
200/200 [==============================] - 0s 1ms/step - loss: 2171.5994 - mse: 2171.5994 - mae: 27.8627
Epoch 17/100
200/200 [==============================] - 0s 2ms/step - loss: 2235.6453 - mse: 2235.6453 - mae: 28.2193
Epoch 18/100
200/200 [==============================] - 0s 1ms/step -

...

222/222 [==============================] - 1s 2ms/step - loss: 2458.9883 - mse: 2458.9883 - mae: 25.5516
Epoch 3/100
222/222 [==============================] - 0s 2ms/step - loss: 2474.0132 - mse: 2474.0132 - mae: 25.9681
Epoch 4/100
222/222 [==============================] - 0s 2ms/step - loss: 2415.8987 - mse: 2415.8987 - mae: 25.7148
Epoch 5/100
222/222 [==============================] - 1s 3ms/step - loss: 2466.0166 - mse: 2466.0166 - mae: 25.4113
Epoch 6/100
222/222 [==============================] - 0s 2ms/step - loss: 2465.0293 - mse: 2465.0293 - mae: 25.7258
Epoch 7/100
222/222 [==============================] - 1s 2ms/step - loss: 2446.0969 - mse: 2446.0969 - mae: 25.6306
Epoch 8/100
222/222 [==============================] - 0s 2ms/step - loss: 2438.3091 - mse: 2438.3091 - mae: 25.7388
Epoch 9/100
222/222 [==============================] - 1s 3ms/step - loss: 2454.2793 - mse: 2454.2793 - mae: 25.5570
Epoch 10/100
222/222 [==============================] - 1s 3ms/step - loss: 

222/222 [==============================] - 0s 2ms/step - loss: 2415.1538 - mse: 2415.1538 - mae: 25.6332
Epoch 71/100
222/222 [==============================] - 1s 2ms/step - loss: 2436.9937 - mse: 2436.9937 - mae: 25.5410
Epoch 72/100
222/222 [==============================] - 1s 3ms/step - loss: 2430.6929 - mse: 2430.6929 - mae: 25.4095
Epoch 73/100
222/222 [==============================] - 1s 3ms/step - loss: 2429.8264 - mse: 2429.8264 - mae: 25.3079
Epoch 74/100
222/222 [==============================] - 0s 2ms/step - loss: 2420.3672 - mse: 2420.3672 - mae: 25.3246
Epoch 75/100
222/222 [==============================] - 0s 2ms/step - loss: 2430.3542 - mse: 2430.3542 - mae: 25.4458
Epoch 76/100
222/222 [==============================] - 0s 1ms/step - loss: 2423.7561 - mse: 2423.7561 - mae: 25.4909
Epoch 77/100
222/222 [==============================] - 0s 1ms/step - loss: 2389.7522 - mse: 2389.7522 - mae: 25.7390
Epoch 78/100
222/222 [==============================] - 0s 1ms/step -

244/244 [==============================] - 0s 1ms/step - loss: 2304.9211 - mse: 2304.9211 - mae: 25.3391
Epoch 40/100
244/244 [==============================] - 0s 1ms/step - loss: 2293.4294 - mse: 2293.4294 - mae: 25.1192
Epoch 41/100
244/244 [==============================] - 0s 1ms/step - loss: 2305.5276 - mse: 2305.5276 - mae: 24.9646
Epoch 42/100
244/244 [==============================] - 0s 1ms/step - loss: 2336.3320 - mse: 2336.3320 - mae: 25.4314
Epoch 43/100
244/244 [==============================] - 0s 1ms/step - loss: 2329.8899 - mse: 2329.8899 - mae: 25.2618
Epoch 44/100
244/244 [==============================] - 0s 1ms/step - loss: 2311.2537 - mse: 2311.2537 - mae: 25.2748
Epoch 45/100
244/244 [==============================] - 0s 1ms/step - loss: 2324.8557 - mse: 2324.8557 - mae: 25.1980
Epoch 46/100
244/244 [==============================] - 0s 1ms/step - loss: 2282.6318 - mse: 2282.6318 - mae: 25.3449
Epoch 47/100
244/244 [==============================] - 0s 1ms/step -

266/266 [==============================] - 0s 1ms/step - loss: 2440.4512 - mse: 2440.4512 - mae: 25.8164
Epoch 9/100
266/266 [==============================] - 0s 1ms/step - loss: 2380.7451 - mse: 2380.7451 - mae: 25.6648
Epoch 10/100
266/266 [==============================] - 0s 1ms/step - loss: 2398.6787 - mse: 2398.6787 - mae: 25.5495
Epoch 11/100
266/266 [==============================] - 0s 1ms/step - loss: 2388.4065 - mse: 2388.4065 - mae: 25.1818
Epoch 12/100
266/266 [==============================] - 0s 1ms/step - loss: 2377.9541 - mse: 2377.9541 - mae: 25.7925
Epoch 13/100
266/266 [==============================] - 0s 2ms/step - loss: 2381.2554 - mse: 2381.2554 - mae: 25.5971
Epoch 14/100
266/266 [==============================] - 0s 2ms/step - loss: 2401.5938 - mse: 2401.5938 - mae: 25.6974
Epoch 15/100
266/266 [==============================] - 0s 1ms/step - loss: 2379.3464 - mse: 2379.3464 - mae: 25.8480
Epoch 16/100
266/266 [==============================] - 0s 1ms/step - 

266/266 [==============================] - 0s 2ms/step - loss: 2377.4958 - mse: 2377.4958 - mae: 25.6415
Epoch 78/100
266/266 [==============================] - 1s 2ms/step - loss: 2375.2390 - mse: 2375.2390 - mae: 25.4845
Epoch 79/100
266/266 [==============================] - 1s 2ms/step - loss: 2406.0417 - mse: 2406.0417 - mae: 25.6220
Epoch 80/100
266/266 [==============================] - 1s 2ms/step - loss: 2352.6785 - mse: 2352.6785 - mae: 25.5131
Epoch 81/100
266/266 [==============================] - 1s 2ms/step - loss: 2339.2380 - mse: 2339.2380 - mae: 25.2331
Epoch 82/100
266/266 [==============================] - 1s 2ms/step - loss: 2305.9897 - mse: 2305.9897 - mae: 25.0965
Epoch 83/100
266/266 [==============================] - 1s 2ms/step - loss: 2352.7561 - mse: 2352.7561 - mae: 25.3672
Epoch 84/100
266/266 [==============================] - 1s 2ms/step - loss: 2308.1296 - mse: 2308.1296 - mae: 25.2149
Epoch 85/100
266/266 [==============================] - 1s 2ms/step -

In [ ]:
data.head()

In [ ]:
rmse_cv = []
for i in mse_cv:
    rmse_cv.append(i ** 0.5)
    
results = pd.DataFrame({'rmse_cv':rmse_cv,
              
                        'mae_cv': mae_cv,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor}, index = split_range)

results

In [ ]:
!pip install matplotlib

%matplotlib notebook

import matplotlib.pyplot as plt

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
split_range = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged RMSE on all test set \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], y = rmse_gen, label = 'Overall error')
plt.legend(loc = 'lower right')
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], split_range)
plt.tight_layout()
plt.savefig('Split_number_2_all_RMSE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged RMSE on spike regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], y =rmse_spi, label = 'Spike regions', color = 'orange')
plt.legend(loc = 'lower right')
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], split_range)
plt.tight_layout()
plt.savefig('Split_number_2_spike_RMSE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged RMSE on normal regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], y =rmse_nor, label = 'Normal regions', color = 'green')
plt.legend(loc = 'lower right')
plt.ylabel('RMSE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], split_range)
plt.tight_layout()
plt.savefig('Split_number_2_nomal_RMSE.png')

In [ ]:
plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged MAE on all test set \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], y = mae_gen, label = 'Overall error')
plt.legend(loc = 'lower right')
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], split_range)
plt.tight_layout()
plt.savefig('Split_number_2_all_MAE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged MAE on spike regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], y =mae_spi, label = 'Spike regions', color = 'orange')
plt.legend(loc = 'lower right')
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], split_range)
plt.tight_layout()
plt.savefig('Split_number_2_spike_MAE.png')

plt.figure(figsize=(7,3.5))
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.title('Averaged MAE on normal regions \n for different split numbers')
plt.scatter(x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], y = mae_nor, label = 'Normal regions', color = 'green')
plt.legend(loc = 'lower right')
plt.ylabel('MAE (£/MWh)')
plt.xlabel('Split number')
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], split_range)
plt.tight_layout()
plt.savefig('Split_number_2_normal_MAE.png')